# Baseline Model Selection: Choosing the XGBoost Classifier for Initial Hyperparameter Assessment
* In this notebook, we evaluate the baseline performance of several classification models, including Random Forest, Logistic Regression, and XGBoost. Our goal is to identify the most promising model based on key metrics such as Incremental Response Rate (IRR) and Net Incremental Revenue (NIR).

* Given the time constraints and the results observed, we have selected the XGBoost classifier as our model of choice for further hyperparameter tuning. This selection will serve as a foundation for optimizing the model's performance in subsequent analyses.

In [1]:
# Import necessary libraries for numerical operations, data manipulation, and machine learning
import numpy as np
import pandas as pd
import optuna
import xgboost as xgb
from xgboost import XGBClassifier

# Import performance metrics and model selection utilities from scikit-learn
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    make_scorer,
)
from sklearn.model_selection import cross_val_score, train_test_split

/Users/spd/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the training dataset
train_data = pd.read_csv("../data/training.csv")

# Load the test dataset
test_data = pd.read_csv("../data/Test.csv")

# Display the first few rows of the training dataset
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044331,-0.385883,1,1,2,2


In [3]:
# Define Group A as g1 (those who did not receive the promotion)
g1 = train_data[train_data.Promotion == "No"]

# Define Group B as g2 (those who received the promotion)
g2 = train_data[train_data.Promotion == "Yes"]

In [4]:
# Split the dataset into training and testing sets, only from group 2 (those who received the promotion)
X = g2[["V1", "V2", "V3", "V4", "V5", "V6", "V7"]]  # Features
y = g2["purchase"]  # Target variable

# Perform the train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42  # 20% test size, random state for reproducibility
)

In [5]:
# Define a promotion strategy function
def promotion_strategy(df, model):
    """
    Predicts purchases for the given DataFrame and converts them to "Yes" or "No" format as promotions.

    Args:
        df (pd.DataFrame): DataFrame with columns V1 - V7.
        model: A trained model used for predictions.

    Returns:
        np.array: A 1-D Numpy array with values 'Yes' or 'No' indicating whether or not an individual should receive a promotion based on the purchase prediction.

    Example:
        >>> promotion_strategy(df)
        array(['Yes', 'Yes', 'No'])

        Input: df
            V1   V2   V3    V4  V5  V6  V7
            2    30   -1.1  1   1   3   2
            3    32   -0.6  2   3   2   2
            2    30   0.13  1   1   4   2

        Output: promotion
            array(['Yes', 'Yes', 'No']) indicates the first two users would receive the promotion and the last should not.
    """
    arr = model.predict(df)

    # Use numpy.where to replace 1 with "Yes", else with "No"
    promotion = np.where(arr == 1, "Yes", "No")

    return promotion

# Function to score the promotion strategy
def score(df, promo_pred_col="Promotion"):
    """
    Calculates the Incremental Response Rate (IRR) and Net Incremental Revenue (NIR).

    Args:
        df (pd.DataFrame): DataFrame containing the predictions and actual purchases.
        promo_pred_col (str): The column name for promotion predictions.

    Returns:
        tuple: IRR and NIR values.
    """
    n_treat = df.loc[df[promo_pred_col] == "Yes", :].shape[0]
    n_control = df.loc[df[promo_pred_col] == "No", :].shape[0]
    n_treat_purch = df.loc[df[promo_pred_col] == "Yes", "purchase"].sum()
    n_ctrl_purch = df.loc[df[promo_pred_col] == "No", "purchase"].sum()
    irr = n_treat_purch / n_treat - n_ctrl_purch / n_control
    nir = 10 * n_treat_purch - 0.15 * n_treat - 10 * n_ctrl_purch

    return (irr, nir)

# Function to test the promotion strategy on the test dataset
def test_results(promotion_strategy, model):
    """
    Tests the promotion strategy on the test dataset and calculates IRR and NIR.

    Args:
        promotion_strategy (function): The promotion strategy function.
        model: The trained model used for predictions.

    Returns:
        tuple: IRR and NIR values for the test dataset.
    """
    # Load test dataset
    test_data = pd.read_csv("../data/Test.csv")
    df = test_data[["V1", "V2", "V3", "V4", "V5", "V6", "V7"]]
    promos = promotion_strategy(df, model)
    score_df = test_data.iloc[np.where(promos == "Yes")]
    irr, nir = score(score_df)

    return irr, nir

In [6]:
# Define Optuna model tuning function
def objective(trial, scale_pos_weight, scoring):
    """
    Objective function for Optuna to optimize XGBoost hyperparameters.

    Args:
        trial (optuna.Trial): A trial object for hyperparameter suggestions.
        scale_pos_weight (float): Weight for balancing imbalanced dataset.
        scoring (str): Scoring metric for cross-validation.

    Returns:
        float: Mean cross-validation score.
    """
    # Define the hyperparameters to be tuned
    params = {
        "verbosity": 0,  # Verbosity level
        "max_depth": trial.suggest_int("max_depth", 1, 7),  # Maximum tree depth
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),  # Learning rate
        "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
        "gamma": trial.suggest_float("gamma", 0, 1),  # Minimum loss reduction required to make a further partition
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),  # Subsample ratio of the training instances
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),  # Subsample ratio of columns when constructing each tree
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0),  # L1 regularization term on weights
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0),  # L2 regularization term on weights
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 300),  # Minimum sum of instance weight needed in a child
    }


    # Create the XGBoost model with the suggested hyperparameters
    model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,  # Balance the positive and negative weights
        n_jobs=-1,  # Use all available cores
        random_state=42,  # Set a random state for reproducibility
        **params  # Unpack the parameters dictionary
    )

    # Perform cross-validation and calculate the mean score
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scoring)

    return scores.mean()


In [7]:
# Define model tuning function for testing different scoring methods
def model_tuning(scoring, n_trials=2):
    """
    Tunes XGBoostClassifier with the specified scoring method and saves results to the 'results' dictionary.

    Args:
        scoring (str or make_scorer object): Scoring method. Run help(cross_val_score) for more info.
        n_trials (int, optional): Number of trials for hyperparameter optimization. Defaults to 2.
    """
    # Only 1.7% purchase rate in group 2, so handle the imbalanced training dataset
    scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1]) + 5

    # Create an Optuna study for hyperparameter tuning
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective(trial, scale_pos_weight, scoring), n_trials=n_trials
    )

    # Get the best parameters and the best score from the study
    best_params = study.best_params
    score = study.best_value

    # Train the tuned XGBoost model with the best parameters
    tuned_xgb_clf = XGBClassifier(
        scale_pos_weight=scale_pos_weight, random_state=42, **best_params
    )
    tuned_xgb_clf.fit(X_train, y_train)

    # Get the number of promotions in the test dataset
    df = test_data[["V1", "V2", "V3", "V4", "V5", "V6", "V7"]]
    promotions = promotion_strategy(df, tuned_xgb_clf)
    n_promotions = promotions[promotions == "Yes"].shape[0]

    # Model evaluation using test_results function
    irr, nir = test_results(promotion_strategy, tuned_xgb_clf)

    # Save each result to the 'results' dictionary
    results["scoring"].append(scoring)
    results["score"].append(score)
    results["irr"].append(irr)
    results["nir"].append(nir)
    results["n_promotions"].append(n_promotions)
    results["n_trials"].append(n_trials)
    results["best_params"].append(best_params)


In [8]:
# Define treat score function
def treat_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()  # Get confusion matrix values
    return tp / (tp + fn + fp)  # Calculate treat score

# Define irr score function
def irr_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()  # Get confusion matrix values
    return tp / (tp + fp) - fn / (fn + tn)  # Calculate Incremental Response Rate (IRR)

# Define nir score function
def nir_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()  # Get confusion matrix values
    return (10 * tp - 0.15 * (tp + fp)) - 10 * fn  # Calculate Net Incremental Revenue (NIR)

# Combine IRR score and NIR score
def irr_nir_score(y_true, y_pred):
    irr = irr_score(y_true, y_pred)  # Calculate IRR
    nir = nir_score(y_true, y_pred)  # Calculate NIR
    return irr * nir  # Return the product of IRR and NIR

In [9]:
# Define scoring methods to evaluate model performance
scorers = [
    "accuracy",  # Standard accuracy score
    "roc_auc",   # Area Under the Receiver Operating Characteristic Curve
    "recall",    # Recall score
    "precision",  # Precision score
    "f1",        # F1 score (harmonic mean of precision and recall)
    make_scorer(treat_score),  # Custom treat score
    make_scorer(irr_score),     # Custom Incremental Response Rate (IRR)
    make_scorer(nir_score),     # Custom Net Incremental Revenue (NIR)
    make_scorer(irr_nir_score), # Custom combined IRR and NIR score
]

# Initialize results dictionary to store tuning outcomes
results = {
    "scoring": [],
    "score": [],
    "irr": [],
    "nir": [],
    "n_promotions": [],
    "n_trials": [],
    "best_params": [],
}

# Loop through each scoring method to perform model tuning
for scorer in scorers:
    model_tuning(scorer, n_trials=500)  # Tune model with the specified scorer


[I 2024-07-20 15:36:56,057] A new study created in memory with name: no-name-c14d634f-920e-4329-9845-5c053fdf578f
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 15:36:56,812] Trial 0 finished with value: 0.6347702499400301 and parameters: {'max_depth': 3, 'learning_rate': 0.05025038674674931, 'n_estimators': 270, 'gamma': 0.6110590266267906, 'subsample': 0.7688948216744244, 'colsample_bytree': 0.9816618590181987, 'reg_alpha': 6.094478115567044, 'reg_lambda': 9.640508446904276, 'min_child_weight': 29}. Best is trial 0 with value: 0.6347702499400301.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be give

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 15:37:04,045] Trial 12 finished with value: 0.7520285752204303 and parameters: {'max_depth': 7, 'learning_rate': 0.08670489121516067, 'n_estimators': 170, 'gamma': 0.7773758109212625, 'subsample': 0.8582517551695017, 'colsample_bytree': 0.5134429333085416, 'reg_alpha': 9.912132471342849, 'reg_lambda': 7.301747799073649, 'min_child_weight': 91}. Best is trial 3 with value: 0.8204833927489694.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 15:37:11,477] Trial 24 finished with value: 0.717506074576116 and parameters: {'max_depth': 7, 'learning_rate': 0.0809484328265698, 'n_estimators': 140, 'gamma': 0.7359444626572734, 'subsample': 0.7184696771191952, 'colsample_bytree': 0.5764243306449819, 'reg_alpha': 7.425773276021035, 'reg_lambda': 5.930820087786149, 'min_child_weight': 157}. Best is trial 3 with value: 0.8204833927489694.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators


/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 15:37:19,684] Trial 36 finished with value: 0.8345284551936054 and parameters: {'max_depth': 6, 'learning_rate': 0.06040720690408897, 'n_estimators': 220, 'gamma': 0.3966063127985418, 'subsample': 0.9367134284432094, 'colsample_bytree': 0.9527918450231909, 'reg_alpha': 6.331135442926368, 'reg_lambda': 2.379004605318553, 'min_child_weight': 15}. Best is trial 36 with value: 0.8345284551936054.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimator

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 15:37:30,762] Trial 48 finished with value: 0.4741375294566287 and parameters: {'max_depth': 1, 'learning_rate': 0.042447636478280606, 'n_estimators': 260, 'gamma': 0.21805846852927763, 'subsample': 0.9583179919088816, 'colsample_bytree': 0.9381454823112135, 'reg_alpha': 8.594426128576261, 'reg_lambda': 0.04414353197057719, 'min_child_weight': 60}. Best is trial 43 with value: 0.8472750428516441.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estim

...

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 16:15:51,610] Trial 448 finished with value: 1.5502775190708717 and parameters: {'max_depth': 1, 'learning_rate': 0.04655531807105268, 'n_estimators': 160, 'gamma': 0.3270775491607275, 'subsample': 0.7893650266780027, 'colsample_bytree': 0.894340879718674, 'reg_alpha': 9.481978518655675, 'reg_lambda': 6.551642224601944, 'min_child_weight': 278}. Best is trial 308 with value: 2.432660511926075.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimato

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 16:15:55,991] Trial 460 finished with value: -2.011311583033474 and parameters: {'max_depth': 4, 'learning_rate': 0.04432081105941054, 'n_estimators': 130, 'gamma': 0.32795703793982134, 'subsample': 0.783366084467181, 'colsample_bytree': 0.864891079919818, 'reg_alpha': 9.612803604111736, 'reg_lambda': 6.136345244973298, 'min_child_weight': 150}. Best is trial 308 with value: 2.432660511926075.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimato

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 16:15:59,538] Trial 472 finished with value: 2.031448766078016 and parameters: {'max_depth': 1, 'learning_rate': 0.043639665868892535, 'n_estimators': 140, 'gamma': 0.2875549382322652, 'subsample': 0.7393071947005043, 'colsample_bytree': 0.8718824416195816, 'reg_alpha': 9.670990247098379, 'reg_lambda': 7.008030812457872, 'min_child_weight': 262}. Best is trial 308 with value: 2.432660511926075.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimat

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 16:16:03,302] Trial 484 finished with value: 1.8736698892407613 and parameters: {'max_depth': 1, 'learning_rate': 0.05088889754937564, 'n_estimators': 150, 'gamma': 0.183751931871593, 'subsample': 0.7785954299408223, 'colsample_bytree': 0.8834946006418894, 'reg_alpha': 9.793522122334078, 'reg_lambda': 6.871887022077549, 'min_child_weight': 261}. Best is trial 308 with value: 2.432660511926075.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimato

/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estimators
[I 2024-07-20 16:16:06,844] Trial 496 finished with value: 2.268652631027158 and parameters: {'max_depth': 1, 'learning_rate': 0.043111205192459784, 'n_estimators': 140, 'gamma': 0.37347269956761964, 'subsample': 0.7663985389264658, 'colsample_bytree': 0.9269207619947265, 'reg_alpha': 9.838765671677242, 'reg_lambda': 6.5160887271108034, 'min_child_weight': 258}. Best is trial 308 with value: 2.432660511926075.
/var/folders/pc/0f0x0j9j6tdd98b7wsgpg8xc0000gn/T/ipykernel_39548/3940685021.py:19: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  "n_estimators": trial.suggest_int("n_estimators", 50, 300, 10),  # Number of estim

In [10]:
# Convert results dictionary into a DataFrame for better visualization and analysis
results_df = pd.DataFrame(results)

results_df

,scoring,score,irr,nir,n_promotions,n_trials,best_params
0,accuracy,0.947774,0.015825,10.25,2116,500,"{'max_depth': 7, 'learning_rate': 0.0977444529..."
1,roc_auc,0.653748,0.020099,464.70,18739,500,"{'max_depth': 4, 'learning_rate': 0.0100143392..."
2,recall,0.855563,0.014231,-107.90,28355,500,"{'max_depth': 1, 'learning_rate': 0.0229251660..."
3,precision,0.026520,0.019668,394.45,17401,500,"{'max_depth': 3, 'learning_rate': 0.0325975613..."
4,f1,0.050820,0.019597,416.55,18553,500,"{'max_depth': 3, 'learning_rate': 0.0295476606..."
5,"make_scorer(treat_score, response_method='pred...",0.025944,0.019763,415.75,17907,500,"{'max_depth': 3, 'learning_rate': 0.0193394409..."
6,"make_scorer(irr_score, response_method='predict')",0.017762,0.020606,519.65,19101,500,"{'max_depth': 3, 'learning_rate': 0.0183463400..."
7,"make_scorer(nir_score, response_method='predict')",132.760000,0.017863,321.70,23166,500,"{'max_depth': 1, 'learning_rate': 0.0736641572..."
8,"make_scorer(irr_nir_score, response_method='pr...",2.432661,0.018437,394.30,23361,500,"{'max_depth': 1, 'learning_rate': 0.0463310264..."


In [13]:
# save baseline tuning results as a CSV file
results_df.to_csv("../reports/baselines/metric_baseline.csv", index=False)

## Conclusion and Next Steps

Our evaluation of the XGBoost classifier across various metrics has revealed valuable insights. Although the accuracy metric achieved a notable value of **0.944**, the class imbalance in our data necessitates a stronger emphasis on metrics such as **ROC AUC**, **F1**, and our custom score metrics. Instead of merely focusing on these scores, our primary objective is to maximize the **Net Incremental Response (NIR)** to drive profit.

Given the results, we will concentrate on the top three metrics—**ROC AUC**, **F1**, and **IRR (Incremental Response Rate)** score. These metrics provide a comprehensive view of our model's performance and align closely with our business goals. In the refinement phase, we will optimize these metrics by fine-tuning hyperparameters within the identified ranges. This targeted approach is designed to improve the overall effectiveness of our model and better address the complexities of our dataset.
